In [ ]:
# 데이터 출처: https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv
# 챗봇 트레이닝용 문답 페어 11,876개
# 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from datetime import datetime
import random
device = torch.device("cuda:1")
# device = torch.device("cpu")

In [ ]:
# 모델, 토크나이저 불러오기
from transformers import DistilBertModel
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')
dimodel = distilbert_model.to(device)

from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')
vocab = tokenizer.get_vocab()

print(len(vocab))
print(tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]"))
tokenizer.convert_tokens_to_ids(['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]'])

In [ ]:
# PREPOCESSING
class diBERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, tokenizer, max_len):
        def ditransform(sent):
            sent_tok = [2] + tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)) + [3]
            token_ids = sent_tok + [1]*(max_len - len(sent_tok))
            return np.array(token_ids, dtype='int32'), np.array(len(sent_tok), dtype='int32')

        self.sentences = [ditransform(i[sent_idx]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    def __len__(self):
        return (len(self.labels))

class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0, path ='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_acc_max = -np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_acc, model):

        score = val_acc

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'Validation acc decreased ({self.val_acc_max:.6f} --> {val_acc:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path) # 싱글 GPU
        # torch.save(model.module.state_dict(), self.path) # 멀티 GPU

        self.val_acc_max = val_acc